# **Analysis of Launch Site Locations Using Interactive Visuals**


In [4]:
!pip install folium
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=3e807ecff19b89c82f0987f6d83c7a818f0d6e2c2a31bf941e43946b767a564d
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [5]:
import folium
import wget
import pandas as pd

In [6]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

## Marking all launch sites on a map


Spacex dataset with latitude and longitude coordinates for the launch sites

In [10]:
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
df=pd.read_csv(spacex_csv_file)
df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [11]:
launch_sites=df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites=launch_sites.groupby(['Launch Site'], as_index=False).first()
launch_sites=launch_sites[['Launch Site', 'Lat', 'Long']]
launch_sites

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Creating a map of all launch site locations


In [22]:
cape_canaveralLC=[28.562302,-80.577356]
cape_canaveralSLC=[28.563197,-80.576820]
kennedy_spacecenter=[28.573255,-80.646895]
vandenberg_airforce=[34.632834,-120.610745]
site_map=folium.Map(location=cape_canaveralLC, zoom_start=4)
circle=folium.Circle(cape_canaveralLC, radius=100, color='#d35400', fill=True).add_child(folium.Popup('Cape Canaveral Space Force Station(Launch Complex 40)'))
marker=folium.map.Marker(cape_canaveralLC,icon=DivIcon(icon_size=(10,10),icon_anchor=(0,0),html='<div style="font-size: 10; color:#d35400;"><b>%s</b></div>' % 'CCAFS LC-40',))
site_map.add_child(circle)
site_map.add_child(marker)
circle2=folium.Circle(cape_canaveralSLC, radius=100, color='#d45600', fill=True).add_child(folium.Popup('Cape Canaveral Space Force Station(Space Launch Complex 40)'))
marker2=folium.map.Marker(cape_canaveralSLC,icon=DivIcon(icon_size=(10,10),icon_anchor=(0,0),html='<div style="font-size: 10; color:#d45600;"><b>%s</b></div>' % 'CCAFS SLC-40',))
site_map.add_child(circle2)
site_map.add_child(marker2)
circle3=folium.Circle(kennedy_spacecenter, radius=1000, color='#d62300', fill=True).add_child(folium.Popup('Kennedy Space Center(Launch Complex 39A)'))
marker3=folium.map.Marker(kennedy_spacecenter,icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0),html='<div style="font-size: 12; color:#d62300;"><b>%s</b></div>' % 'KSC LC-39A',))
site_map.add_child(circle3)
site_map.add_child(marker3)
circle4=folium.Circle(vandenberg_airforce, radius=1000, color='#d24500', fill=True).add_child(folium.Popup('Vandenberg Air Force Center(Space Launch Complex 4)'))
marker4=folium.map.Marker(vandenberg_airforce,icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0),html='<div style="font-size: 12; color:#d24500;"><b>%s</b></div>' % 'VAFB SLC-4E',))
site_map.add_child(circle4)
site_map.add_child(marker4)

# Marking the successful and failed launches for each site


Creating a function that turns successful launches green and unsuccesful launches red

In [24]:
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
df['marker_color']=df['class'].apply(assign_marker_color)

Adding marker clusters to the map to signify which launch sites have high success rates

In [29]:
marker_cluster=MarkerCluster()
site_map.add_child(marker_cluster)

for index, record in df.iterrows():
    marker=folium.Marker([record['Lat'],record['Long']],icon=folium.Icon(color='white', icon_color=record['marker_color']))
    marker_cluster.add_child(marker)

site_map

From this map, we can see that the Kennedy Space Center had the highest success rate

# Calculating the distances between each launch site to its proximities


Adding a `MousePosition` on the map to get coordinates of any points of interests, such as railways, highways, and coasts


In [30]:
formatter="function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [31]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R=6373.0

    lat1=radians(lat1)
    lon1=radians(lon1)
    lat2=radians(lat2)
    lon2=radians(lon2)

    dlon=lon2 - lon1
    dlat=lat2 - lat1

    a=sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c=2 * atan2(sqrt(a), sqrt(1 - a))

    distance=R * c
    return distance

Distance between coastline and Cape Canaveral Station

In [33]:
distance_coastline= calculate_distance(28.562302,-80.577356,28.5639,-80.56791)
distance_coastline

0.9397625538160118

Creating a marker to represent this distance

In [38]:
distance_marker=folium.Marker([28.5639,-80.56791], icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0),html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),))
site_map.add_child(distance_marker)
lines=folium.PolyLine(locations=[[28.562302,-80.577356],[28.5639,-80.56791]], weight=1)
site_map.add_child(lines)

All launch sites are close to the coastline and mostly away from highways and railroads, with the exception of the Kennedy Launch Complex which is next to the Kennedy Parkway and a railroad